In [14]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, make_scorer

# Load the training set (gene expression data for different perturbations)
train_df = pd.read_csv("data/train_set.csv", index_col=0)
# Load the test set (perturbations to predict)
test_df = pd.read_csv("data/test_set.csv", header=None, names=["perturbation"])

# Find columns in the training set that correspond to single-gene perturbations (ending with '+ctrl')
single_pert_cols = [col for col in train_df.columns if "+ctrl" in col]
# Map gene name to its single-gene perturbation column
single_gene_map = {col.split("+")[0]: col for col in single_pert_cols}

# Training Data
X_train, Y_train = [], []

# For each double-gene perturbation column, build input and output for training
for col in train_df.columns:
    if "+ctrl" in col:
        continue  # skip single-gene columns
    genes = col.split("+")
    if len(genes) != 2:
        continue
    gA, gB = genes
    # Only use if both genes have single-gene data
    if gA in single_gene_map and gB in single_gene_map:
        # Get expression profiles for each gene's single-gene perturbation
        expr_A = train_df[single_gene_map[gA]].values
        expr_B = train_df[single_gene_map[gB]].values
        # Concatenate the two profiles to form the input
        X_train.append(np.concatenate([expr_A, expr_B]))
        # The output is the expression profile for the double-gene perturbation
        Y_train.append(train_df[col].values)

# Convert lists to numpy arrays for model training
X_train = np.array(X_train)
Y_train = np.array(Y_train)

# Tuning Ridge Regression using Cross-Validation
# Define RMSE as the scoring metric (since we are being evaluated by this)
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Use neg RMSE for scoring (since GridSearchCV maximizes the score)
scorer = make_scorer(rmse, greater_is_better=False)
ridge = Ridge()
# Testing with different values of alpha (regularization strength)
grid = GridSearchCV(ridge, {"alpha": [0.01, 0.1, 1, 10, 100, 200, 500, 10000]}, scoring=scorer, cv=5)
# Use MultiOutputRegressor to fit one regressor per output gene
multi_grid = MultiOutputRegressor(grid)
multi_grid.fit(X_train, Y_train)

# Get the best alpha for each output gene and average them
best_alphas = [est.best_params_["alpha"] for est in multi_grid.estimators_]
best_alpha = np.mean(best_alphas)
print(f"Average best alpha from CV: {best_alpha}")

# Retrain model with best alpha
final_model = MultiOutputRegressor(Ridge(alpha=best_alpha))
final_model.fit(X_train, Y_train)

# Make Predictions
predictions = []
# For each test perturbation, predict the gene expression profile
for perturb in test_df["perturbation"]:
    gA, gB = perturb.split("+")

    #function to get expression profile for a gene
    def get_expr(gene):
        if gene in single_gene_map:
            # Use single-gene perturbation if available
            return train_df[single_gene_map[gene]].values
        # Otherwise average all columns where this gene is present
        cols = [col for col in train_df.columns if gene in col.split("+")]
        return train_df[cols].mean(axis=1).values if cols else np.zeros(train_df.shape[0])

    # Build input vector for the model
    expr_A = get_expr(gA)
    expr_B = get_expr(gB)
    input_vec = np.concatenate([expr_A, expr_B]).reshape(1, -1)
    # Predict expression for all genes
    pred = final_model.predict(input_vec).flatten()

    # Store predictions and clip negative values to zero (since gene expressions cant be negative)
    for i, gene in enumerate(train_df.index):
        predictions.append((gene, perturb, max(0, pred[i])))  # clipping to 0

# Save predictions to CSV
prediction_df = pd.DataFrame(predictions, columns=["gene", "perturbation", "expression"])
prediction_df.to_csv("prediction/prediction.csv", index=False)


Average best alpha from CV: 7126.3200099999995


In [15]:
avg_rmse = np.mean([-est.best_score_ for est in multi_grid.estimators_])
print(f"Average CV RMSE: {avg_rmse:.4f}")


Average CV RMSE: 0.3740
